In [ ]:
import pandas as pd
from pathlib import Path
from typing import Dict, List
#import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import TimeSeriesSplit


# TODO: Try windsorizing the data for beter results in future to eliminate the extereme cases
# TODO: Add cross-sectional ranking within an asset class to give the model some insight about asset rankings.

## ------------------
## Load data
## ------------------
DATA_DIR = Path('.')

pairs = pd.read_csv(DATA_DIR/'target_pairs.csv')
raw = pd.read_csv(DATA_DIR/'train.csv')
labels = pd.read_csv(DATA_DIR/'train_labels.csv')

pairs['target_id'] = pairs['target'].str.split('_').str[1].astype(int)
pairs['legs'] = pairs['pair'].str.split(' - ')
pairs['type'] = pairs['legs'].apply(lambda x: 'pair' if len(x) == 2 else 'single').astype(str)

target_instruments = list(set([leg for legs in pairs['legs'] for leg in legs]))

# Price series of target instruments
prices = pd.melt(raw, id_vars="date_id", value_vars=target_instruments)
prices.rename(columns={"variable": "instrument", "value": "price"}, inplace=True)
prices = prices.sort_values(["date_id", "instrument"]).reset_index(drop=True)

## ------------------
## Feature engineering
## ------------------
LAGS = [1, 2, 3]
SMA_WINDOWS = [3, 5, 10, 20]
EMA_WINDOWS = [3, 5, 10, 20]
VOL_WINDOWS = [3, 5, 10, 20]
RSI_WINDOW = 14

def compute_per_instrument_features(prices: pd.DataFrame) -> pd.DataFrame:

    df = prices.copy()

    df["log_price"] = np.log(df["price"])
    df["ret_1"] = df.groupby("instrument")["log_price"].diff(1)

    # Lag feature 
    for lag in LAGS:
        df[f"lag_{lag}"] = df.groupby("instrument")["ret_1"].shift(lag)

    # Simple moving average to capture trends
    for window in SMA_WINDOWS:
        df[f"SMA_{window}"] = df.groupby("instrument")["ret_1"].transform(
            lambda x: x.shift(1).rolling(window=window, min_periods=window).mean())

    # Exponential moving averege to capture decaying impact
    for window in EMA_WINDOWS:
        df[f"EMA_{window}"] = df.groupby("instrument")["ret_1"].transform(
            lambda x: x.shift(1).ewm(span=window, min_periods=window).mean())

    # Rolling volitility to capture risk
    for window in VOL_WINDOWS:
        df[f"VOL_{window}"] = df.groupby("instrument")["ret_1"].transform(
            lambda x: x.shift(1).rolling(window=window, min_periods=window).std())

    # Relative Strength Index (RSI)
    # Over 70 hints that the stock is overbought, and below 30 means oversold.
    df["up"] = df["ret_1"].clip(lower=0)
    df["down"] = -df["ret_1"].clip(upper=0)
    df["avg_gain"] = df.groupby("instrument")["up"].transform(
        lambda x: x.shift(1).rolling(RSI_WINDOW).mean())
    df["avg_loss"] = df.groupby("instrument")["down"].transform(
        lambda x: x.shift(1).rolling(RSI_WINDOW).mean()).replace(0, np.nan)
    df["RS"] = df["avg_gain"] / df["avg_loss"]
    df["RSI"] = 100 - (100 / (1 + df["RS"]))

    # Cyclic day encoding via one-hot
    df["DOW"] = df["date_id"] % 5
    df = pd.get_dummies(df, columns=["DOW"], prefix="DOW", dtype=int)

    return df

base_features = compute_per_instrument_features(prices)

## ------------------
## Feature store
## ------------------
keep_cols = ["date_id", "instrument", "price", "log_price", "ret_1"]
keep_cols += [f"lag_{lag}" for lag in LAGS]
keep_cols += [f"SMA_{window}" for window in SMA_WINDOWS]
keep_cols += [f"EMA_{window}" for window in EMA_WINDOWS]
keep_cols += [f"VOL_{window}" for window in VOL_WINDOWS]
keep_cols += ["RSI"]
keep_cols += [f"DOW_{date}" for date in [0, 1, 2, 3, 4]]

feature_store = base_features[keep_cols].copy()
feature_store.set_index(["date_id", "instrument"], inplace=True)
feature_store.sort_index(inplace=True)

## ------------------
## Long format 
## ------------------
for col in labels.columns:
    if col.startswith("target"):
        n = col.split("_")[1]
        labels.rename(columns={f"{col}": f"{n}"}, inplace=True)

# Melt into long format
labels_long = (
    labels.melt(id_vars="date_id")
    .rename(columns={"date_id": "label_date_id", "variable": "target_id", "value": "price"})
)
labels_long = labels_long.astype({"target_id": int})

# Attach pair_type + legs
legs_map: Dict[int, List[str]] = dict(zip(pairs['target_id'], pairs['legs']))
pair_type_map: Dict[int, str] = dict(zip(pairs['target_id'], pairs['type']))

labels_long["pair_type"] = labels_long["target_id"].map(pair_type_map)
labels_long["legs"] = labels_long["target_id"].map(legs_map)

# Effective date gives the first date that features are taken from
MAX_LOOKBACK_WINDOW = max(max(LAGS), max(SMA_WINDOWS), max(EMA_WINDOWS), max(VOL_WINDOWS), RSI_WINDOW)
labels_long["eff_date_id"] = labels_long["label_date_id"] - MAX_LOOKBACK_WINDOW

## ------------------
## Get leg features 
## ------------------
def get_leg_features(label_date_id: int, instrument: str, prefix: str):
    try:
        row = feature_store.loc[(int(label_date_id), instrument)]
    except KeyError:
        return None
    return {f"{prefix}{k}": v for k, v in row.items()}

## ------------------
## Build training matrix
## ------------------
rows = []
diff = [ins for ins in keep_cols if not ins in ["date_id", "instrument", "DOW_0", "DOW_1", "DOW_2", "DOW_3", "DOW_4"]]

for i, label_row in labels_long.iterrows():
    label_date_id = int(label_row["label_date_id"])
    target_id = int(label_row["target_id"])
    y = float(label_row["price"])
    pair_type = label_row["pair_type"]
    eff_date_id = int(label_row["eff_date_id"])
    legs = label_row["legs"]

    base = {
        "label_date_id": label_date_id,
        "target_id": target_id,
        "y": y,
        "eff_date_id": eff_date_id,
        "pair_type_single": 1 if pair_type == "single" else 0,
        "pair_type_pair": 1 if pair_type == "pair" else 0,
    }

    if pair_type == "single":
        leg = legs[0]
        A_features = get_leg_features(label_date_id, leg, prefix="A_")
        if A_features is None:
            continue   
        row = {**base, **A_features}
    else:
        A, B = legs
        A_features = get_leg_features(label_date_id, A, prefix="A_")
        B_features = get_leg_features(label_date_id, B, prefix="B_")
        if (A_features is None) or (B_features is None):
            continue
        row = {**base, **A_features, **B_features}
        for feature in diff:
            a_key = f"A_{feature}"
            b_key = f"B_{feature}"
            row[f"diff_{feature}"] = row[a_key] - row[b_key]

    rows.append(row)

train_matrix = pd.DataFrame(rows)

## ------------------
## Drop all features but lags for testing
## ------------------
train_matrix_onlyLags = train_matrix.drop(columns=train_matrix.filter(regex="EMA|SMA|VOL|DOW|RSI").columns).copy()
train_matrix_onlyLags = train_matrix_onlyLags[train_matrix_onlyLags["eff_date_id"] >= 0]

train_matrix_onlyLags

,label_date_id,target_id,y,eff_date_id,pair_type_single,pair_type_pair,A_price,A_log_price,A_ret_1,A_lag_1,...,B_ret_1,B_lag_1,B_lag_2,B_lag_3,diff_price,diff_log_price,diff_ret_1,diff_lag_1,diff_lag_2,diff_lag_3
20,20,0,-0.000766,0,1,0,66.7859,4.201492,-0.010296,-0.008313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,21,0,-0.022090,1,1,0,66.8456,4.202385,0.000894,-0.010296,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,22,0,-0.039157,2,1,0,66.7944,4.201619,-0.000766,0.000894,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,23,0,0.020702,3,1,0,65.3351,4.179529,-0.022090,-0.000766,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,24,0,-0.012854,4,1,0,62.8262,4.140372,-0.039157,-0.022090,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812803,1912,423,NaN,1892,0,1,9523.5000,9.161518,0.008012,-0.013614,...,-0.001757,-0.006222,-0.032608,0.025462,9472.32,5.226169,0.009769,-0.007392,0.029533,-0.024733
812804,1913,423,-0.141053,1893,0,1,9519.5000,9.161098,-0.000420,0.008012,...,0.023558,-0.001757,-0.006222,-0.032608,9467.10,5.202191,-0.023978,0.009769,-0.007392,0.029533
812805,1914,423,-0.127688,1894,0,1,9533.5000,9.162567,0.001470,-0.000420,...,-0.004399,0.023558,-0.001757,-0.006222,9481.33,5.208060,0.005869,-0.023978,0.009769,-0.007392
812806,1915,423,-0.012187,1895,0,1,9500.5000,9.159100,-0.003467,0.001470,...,0.012193,-0.004399,0.023558,-0.001757,9447.69,5.192399,-0.015660,0.005869,-0.023978,0.009769


In [16]:
train_matrix_onlyLags.columns

Index(['label_date_id', 'target_id', 'y', 'eff_date_id', 'pair_type_single',
       'pair_type_pair', 'A_price', 'A_log_price', 'A_ret_1', 'A_lag_1',
       'A_lag_2', 'A_lag_3', 'B_price', 'B_log_price', 'B_ret_1', 'B_lag_1',
       'B_lag_2', 'B_lag_3', 'diff_price', 'diff_log_price', 'diff_ret_1',
       'diff_lag_1', 'diff_lag_2', 'diff_lag_3'],
      dtype='object')

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

##
## Try for 1 target intially 
##
target_to_predict = 423
df1 = train_matrix_onlyLags[train_matrix_onlyLags["target_id"] == target_to_predict].copy()
df1 = df1.replace([np.inf, -np.inf], np.nan)
df1 = df1[df1["y"].notna()]

exclude_cols = ["label_date_id", "target_id", "y", "eff_date_id", "pair_type_single", "pair_type_pair", 
                "A_price", "A_log_price", "B_price", "B_log_price", "diff_price", "diff_log_price"]
feature_cols = [col for col in df1.columns if col not in exclude_cols]

X = df1[feature_cols].copy()
X = X.apply(pd.to_numeric, errors="coerce")  # Just in case
y = df1["y"].values


In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

feature_importance = pd.DataFrame(0, index=X.columns, columns=["importance"])
rmse_scores= []

for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=list(X.columns))
    dval = xgb.DMatrix(X_val, label=y_val, feature_names=list(X.columns))

    params = {
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "eta": 0.1,
        "max_depth": 3,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "seed": 42,
    }
    # Train
    model = xgb.train(params, dtrain, num_boost_round=500,
                      evals=[(dval, "validation")],
                      early_stopping_rounds=20,
                      verbose_eval=False)

    # Predictions & RMSE
    preds = model.predict(dval)
    rmse = np.sqrt(mean_squared_error(y_val, preds))

    rmse_scores.append(rmse)
    print(f"Fold {fold+1} RMSE: {rmse:.5f}")

    # Feature importance (gain-based)
    fold_importance = model.get_score(importance_type="gain")
    for feat, imp in fold_importance.items():
        feature_importance.loc[feat, "importance"] += imp
